In [15]:
#用來偵測
from docx import Document

def check_table_cells(doc_path):
    doc = Document(doc_path)
    table = doc.tables[0]  # 假設我們要處理的表格是第一個

    # 遍歷表格的行和列
    for row_index, row in enumerate(table.rows):
        for col_index, cell in enumerate(row.cells):
            # 輸出行和列索引及儲存格內容
            print(f"Row: {row_index}, Column: {col_index}, Content: '{cell.text}'")

# 使用示例
doc_path = r"your word news template"
check_table_cells(doc_path)


Row: 0, Column: 0, Content: '天然氣市場'
Row: 0, Column: 1, Content: '天然氣市場'
Row: 0, Column: 2, Content: '單位'
Row: 0, Column: 3, Content: '價格'
Row: 0, Column: 4, Content: '價格'
Row: 0, Column: 5, Content: '價格'
Row: 0, Column: 6, Content: '說      明'
Row: 1, Column: 0, Content: '天然氣市場'
Row: 1, Column: 1, Content: '天然氣市場'
Row: 1, Column: 2, Content: '單位'
Row: 1, Column: 3, Content: '10/11
收盤'
Row: 1, Column: 4, Content: '10/10
收盤'
Row: 1, Column: 5, Content: '+漲/  
-跌'
Row: 1, Column: 6, Content: '說      明'
Row: 2, Column: 0, Content: '天然氣'
Row: 2, Column: 1, Content: '美國
Henry Hub
(期貨)'
Row: 2, Column: 2, Content: '美元/MMBtu'
Row: 2, Column: 3, Content: ''
Row: 2, Column: 4, Content: ''
Row: 2, Column: 5, Content: ''
Row: 2, Column: 6, Content: '。
。
據Baker Hughes週鑽機數報告(10/11發布)，上週美國天然氣鑽機數量為101座，較前一週減少1座，相較於去年同期117座減少16座(-13.7%)。其中，主要產氣盆地鑽機數如下說明：
Haynesville盆地天然氣鑽機數為34座，與前一週持平，相較於去年同期37座減少3座(-8.1%)。
Marcellus盆地天然氣鑽機數為23座，較前一週減少2座(-8.0%)，相較於去年同期29座減少6座(-20.7%)。
'
Row: 3, Column: 0, Content: '天然氣'

In [1]:
import os
import shutil
from datetime import datetime, timedelta
import xlwings as xw
from docx import Document
import win32com.client as win32
import time
from docx.shared import RGBColor, Pt
from docx.oxml.ns import qn

# 設定資料夾路徑
EXCEL_PATH = r"your price data"
TEMPLATE_FOLDER = r"your word news template"
OUTPUT_FOLDER = r"your output folder"

############################################################
def update_word_header(doc_path):
    """更新 Word 檔案的頁首內容"""
    word = win32.gencache.EnsureDispatch('Word.Application')
    word.Visible = True  # 背景運行 Word，方便觀察
    try:
        doc = word.Documents.Open(doc_path)
        today_date = datetime.today().strftime("%Y/%m/%d")

        # 遍歷所有分節，更新每個頁首
        for section in doc.Sections:
            for i, header in enumerate(section.Headers):
                print(f"正在更新第 {i + 1} 頁的頁首")
                header.Range.Text = f"每日天然氣價格及市場摘要報告 ({today_date})"

        doc.Save()
        print("頁首已成功更新")
    except Exception as e:
        print(f"更新頁首時發生錯誤: {e}")
    finally:
        doc.Close()
        word.Quit()
        time.sleep(2)  # 確保 Word 完全退出

############################################################
def get_template_filename():
    day_map = {0: "mon", 1: "tue", 2: "wed", 3: "thr", 4: "fri"}
    today = datetime.today().weekday()
    return f"每日天然氣價格與市場摘要報告_{day_map.get(today)}.docx" if today in day_map else None

def copy_and_rename_template(template_filename):
    today_str = datetime.today().strftime("%Y%m%d")
    src = os.path.join(TEMPLATE_FOLDER, template_filename)
    dest = os.path.join(OUTPUT_FOLDER, f"每日天然氣價格與市場摘要報告_油氣資源組{today_str}.docx")
    shutil.copy(src, dest)
    print(f"已複製模板為: {dest}")
    return dest

def get_latest_prices():
    """從 Excel 中讀取最新價格數據"""
    try:
        app = xw.App(visible=False)
        wb = app.books.open(EXCEL_PATH)
        sheet = wb.sheets[0]

        # 取得最新價格與變動數據
        price_future = sheet.range('M5').value
        price_future_yesterday = sheet.range('M6').value
        diff_future = round(sheet.range('N5').value, 3)
        price_spot = sheet.range('K5').value
        price_spot_yesterday = sheet.range('K6').value
        diff_spot = round(sheet.range('L5').value, 3)
        price_JKM = sheet.range('Q5').value
        price_JKM_yesterday = sheet.range('Q6').value
        diff_JKM = round(sheet.range('R5').value, 3)
        price_TTF = sheet.range('U5').value
        price_TTF_yesterday = sheet.range('U6').value
        diff_TTF = round(sheet.range('V5').value, 3)

        date_yesterday = sheet.range('D5').value
        date_day_before = sheet.range('D6').value

        wb.close()
        app.quit()

        return {
            'price_future': price_future,
            'price_future_yesterday': price_future_yesterday,
            'diff_future': diff_future,
            'price_spot': price_spot,
            'price_spot_yesterday': price_spot_yesterday,
            'diff_spot': diff_spot,
            'price_JKM': price_JKM,
            'price_JKM_yesterday': price_JKM_yesterday,
            'diff_JKM': diff_JKM,
            'price_TTF': price_TTF,
            'price_TTF_yesterday': price_TTF_yesterday,
            'diff_TTF': diff_TTF,
            'date_yesterday':date_yesterday,
            'date_day_before': date_day_before
        }
    except Exception as e:
        print(f"讀取價格資料時發生錯誤: {e}")
        return {}

def format_difference(value):
    """格式化數字，顯示 +/- 符號並保留三位小數"""
    if value > 0:
        return f"+{value:.3f}"
    elif value < 0:
        return f"{value:.3f}"
    else:
        return "0.000"

def set_cell_style(cell, font_color, size, font_name="Arial", bold=True):  
    """設定單元格的字體樣式"""
    for paragraph in cell.paragraphs:
        for run in paragraph.runs:
            run.font.color.rgb = font_color
            run.font.size = size
            run.font.bold = bold  # 設定粗體
            run.font.name = font_name
            run._element.rPr.rFonts.set(qn('w:eastAsia'), '標楷體')  # 中文字體
        paragraph.alignment = 1  # 水平置中

def update_report(doc_path, prices):
    """更新報告內容與格式"""
    from docx.shared import Pt, RGBColor

    doc = Document(doc_path)
    table = doc.tables[0]

    # 定義字體樣式
    blue = RGBColor(47, 84, 150)
    red = RGBColor(225, 0, 0)
    font_size = Pt(12)
    font_name = "Arial"

    # 更新日期標題
    yesterday = prices.get('date_yesterday',"").strftime("%m/%d")
    day_before = prices.get('date_day_before',"").strftime("%m/%d")
    
    # 處理日期儲存格 (1,3) 和 (1,4)
    paragraph = table.cell(1, 3).paragraphs[0]
    paragraph.clear()
    paragraph.add_run(f"{yesterday}").font.size = font_size

    paragraph = table.cell(1, 4).paragraphs[0]
    paragraph.clear()
    paragraph.add_run(f"{day_before}").font.size = font_size

    # 插入今天的價格 (藍色儲存格)
    cells_blue = [(2, 3), (3, 3), (4, 3), (5, 3)]
    for row, col in cells_blue:
        key = ["future", "spot", "JKM", "TTF"][row - 2]
        value = prices.get(f'price_{key}', "")
        table.cell(row, col).text = f"{value:.3f}"
        set_cell_style(table.cell(row, col), blue, font_size, font_name)

    # 插入昨天的價格 (藍色儲存格)
    cells_blue_y = [(2, 4), (3, 4), (4, 4), (5, 4)]
    for row, col in cells_blue_y:
        key = ["future", "spot", "JKM", "TTF"][row - 2]
        value = prices.get(f'price_{key}_yesterday', "")
        table.cell(row, col).text = f"{value:.3f}"
        set_cell_style(table.cell(row, col), blue, font_size, font_name)

    # 插入價差變動 (紅色儲存格)
    cells_red = [(2, 5), (3, 5), (4, 5), (5, 5)]
    for row, col in cells_red:
        key = ["future", "spot", "JKM", "TTF"][row - 2]
        diff_value = prices.get(f'diff_{key}', 0)
        formatted_diff = format_difference(diff_value)
        table.cell(row, col).text = formatted_diff
        set_cell_style(table.cell(row, col), red, font_size, font_name)

    doc.save(doc_path)
    print(f"已更新報告: {doc_path}")

def main():
    template_filename = get_template_filename()
    if not template_filename:
        print("今天不是工作日，無需處理。")
        return

    new_doc_path = copy_and_rename_template(template_filename)
    prices = get_latest_prices()
    if not prices:
        print("未能從 Excel 讀取到價格資料。")
        return

    update_word_header(new_doc_path)
    update_report(new_doc_path, prices)

if __name__ == "__main__":
    main()


已複製模板為: \\Tpfpccpre20175\礦權管理\3_經析-市場行情\00_油資組每日天然氣市場報告\每日天然氣價格與市場摘要報告_油氣資源組20250207.docx
正在更新第 1 頁的頁首
正在更新第 2 頁的頁首
正在更新第 3 頁的頁首
頁首已成功更新
已更新報告: \\Tpfpccpre20175\礦權管理\3_經析-市場行情\00_油資組每日天然氣市場報告\每日天然氣價格與市場摘要報告_油氣資源組20250207.docx


In [2]:
def generate_report_text(prices):
    """根據價格變化自動生成報告文字"""

    # 取得期貨和現貨的價差，並轉換為美分
    diff_future = prices['diff_future'] * 100  # 美分
    diff_spot = prices['diff_spot'] * 100
    future_price = prices['price_future']
    spot_price = prices['price_spot']

    # Henry Hub 的報告文字
    if diff_future > 0 and diff_spot > 0:
        hh_text = f"美國天然氣期貨、現貨價格皆上漲，Henry Hub期貨近月合約價格上漲{diff_future:.1f}美分，收盤{future_price:.3f}美元/MMBtu。Henry Hub現貨平均價格上漲{diff_spot:.1f}美分，收盤{spot_price:.3f}美元/MMBtu。"
    elif diff_future < 0 and diff_spot < 0:
        hh_text = f"美國天然氣期貨、現貨價格皆下跌，Henry Hub期貨近月合約價格下跌{abs(diff_future):.1f}美分，收盤{future_price:.3f}美元/MMBtu。Henry Hub現貨平均價格下跌{abs(diff_spot):.1f}美分，收盤{spot_price:.3f}美元/MMBtu。"
    elif diff_future > 0 and diff_spot < 0:
        hh_text = f"美國天然氣期貨價格上漲、現貨價格下跌，Henry Hub期貨近月合約價格上漲{diff_future:.1f}美分，收盤{future_price:.3f}美元/MMBtu。Henry Hub現貨平均價格下跌{abs(diff_spot):.1f}美分，收盤{spot_price:.3f}美元/MMBtu。"
    else:
        hh_text = f"美國天然氣期貨價格下跌、現貨價格上漲，Henry Hub期貨近月合約價格下跌{abs(diff_future):.1f}美分，收盤{future_price:.3f}美元/MMBtu。Henry Hub現貨平均價格上漲{diff_spot:.1f}美分，收盤{spot_price:.3f}美元/MMBtu。"

    # 取得 JKM 和 TTF 的價差，並轉換為美分
    diff_JKM = prices['diff_JKM'] * 100
    diff_TTF = prices['diff_TTF'] * 100
    jkm_price = prices['price_JKM'] 
    ttf_price = prices['price_TTF'] 

    # JKM 和 TTF 的報告文字
    if diff_JKM > 0 and diff_TTF > 0:
        lng_text = f"日本、歐洲液化天然氣期貨價格皆上漲，日本JKM期貨近月價格上漲{diff_JKM:.1f}美分，收盤{jkm_price:.3f}美元/MMBtu。歐洲TTF期貨近月價格上漲{diff_TTF:.1f}美分至{ttf_price:.3f}美元/MMBtu。"
    elif diff_JKM < 0 and diff_TTF < 0:
        lng_text = f"日本、歐洲液化天然氣期貨價格皆下跌，日本JKM期貨近月價格下跌{abs(diff_JKM):.1f}美分，收盤{jkm_price:.3f}美元/MMBtu。歐洲TTF期貨近月價格下跌{abs(diff_TTF):.1f}美分至{ttf_price:.3f}美元/MMBtu。"
    elif diff_JKM < 0 and diff_TTF > 0:
        lng_text = f"日本液化天然氣價格下跌、歐洲價格上漲，日本JKM期貨近月價格下跌{abs(diff_JKM):.1f}美分至{jkm_price:.3f}美元/MMBtu。歐洲TTF期貨近月價格上漲{diff_TTF:.1f}美分至{ttf_price:.3f}美元/MMBtu。"
    else:
        lng_text = f"日本液化天然氣價格上漲、歐洲價格下跌，日本JKM期貨近月價格上漲{diff_JKM:.1f}美分至{jkm_price:.3f}美元/MMBtu。歐洲TTF期貨近月價格下跌{abs(diff_TTF):.1f}美分至{ttf_price:.3f}美元/MMBtu。"

    return hh_text, lng_text

def update_additional_text(doc_path, hh_text, lng_text):
    """將報告文字插入 (2, 6) 儲存格，並設置為標楷體"""
    doc = Document(doc_path)
    table = doc.tables[0]

    # 插入文字到 (2, 6) 儲存格
    paragraph = table.cell(2, 6).paragraphs[0]
    paragraph.clear()  # 清空現有內容
    run = paragraph.add_run(hh_text + "\n\n" + lng_text)  # 加入報告文字
    run.font.size = Pt(11)  # 設定字體大小
    run.font.name = "標楷體"  # 設定為標楷體
    run._element.rPr.rFonts.set(qn('w:eastAsia'), '標楷體')

    # 儲存文件
    doc.save(doc_path)
    print(f"(2, 6) 儲存格已更新，報告文字插入成功：{doc_path}")

# 取得新的報告文字
hh_text, lng_text = generate_report_text(get_latest_prices())

# 更新 Word 文件的 (2, 6) 儲存格
doc_path = r"your output folder"
update_additional_text(doc_path, hh_text, lng_text)


(2, 6) 儲存格已更新，報告文字插入成功：\\Tpfpccpre20175\礦權管理\3_經析-市場行情\00_油資組每日天然氣市場報告\每日天然氣價格與市場摘要報告_油氣資源組20250207.docx
